# Кластеризация запроса пользователя

Необходимо сделать алгоритм, который кластеризует запросы, выделить с помощью него семантические группы. В итоге по входному списку поисковых запросов должен выдаваться список запросов с кластерами для них по разным семантическим признакам.
Кластеризацию сделать по следующим признакам:
1. по занятости (Фильтр Занятость: подработка, ночная, вечерняя, посменная, вахта и тд.)
2. по должности-лемме (повар, строитель, водитель и тд.)
3. по дополнительному признаку: для инвалидов, для студентов, для школьников, для пенсионеров, для мужчин, для женщин
4. по условиям: с ежедневной оплатой, с проживанием
5. общие фразы про работу (не содержит других признаков)

Одна и та же фраза может попасть в разные кластеры

## Загрузка данных

In [80]:
import pandas as pd
import numpy as np

In [81]:
df = pd.read_csv("data/answers.csv", index_col=0)


In [82]:
df

,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,фарпост работа владивосток,NaN,NaN,NaN,NaN,общая фраза
1,кофейни вакансии,NaN,NaN,NaN,NaN,общая фраза
2,работа разнорабочие часовой,NaN,Рабочий,NaN,NaN,NaN
3,личный водитель на день,на неполный день,Водитель,NaN,NaN,NaN
4,работа от работодателя персональный водитель,NaN,Водитель,NaN,NaN,NaN
...,...,...,...,...,...,...
14240,япония вакансии,NaN,NaN,NaN,NaN,общая фраза
14241,япония работа,NaN,NaN,NaN,NaN,общая фраза
14242,японский язык работа,NaN,NaN,NaN,NaN,общая фраза
14243,яппи вакансии,NaN,NaN,NaN,NaN,общая фраза


## Анализ меток

In [83]:
# видно, что тут задача мультиклассификации, так как меток несколько
for col in  df.drop(columns=['query']).columns:
    print(col)
    print(f"Кол-во непустых: {(~df[col].isna()).sum()}")
    print(f"Количество с комбинациями: {df[col].str.contains(',').sum()}")
    print()

занятость
Кол-во непустых: 1551
Количество с комбинациями: 190

по должности-лемме
Кол-во непустых: 7883
Количество с комбинациями: 431

по дополнительному признаку
Кол-во непустых: 849
Количество с комбинациями: 22

по условиям
Кол-во непустых: 566
Количество с комбинациями: 0

общие фразы
Кол-во непустых: 5223
Количество с комбинациями: 0



In [84]:
col = "по условиям"
df[col].str.split(',').explode().value_counts()

по условиям
с ежедневной оплатой    374
с проживанием           192
Name: count, dtype: int64

## Формирование списка меток

In [85]:
df_mc = df.copy()
for col in df_mc.drop(columns=['query']).columns:
    df_mc[col] = df_mc[col].str.strip().str.lower().str.split(',')
df_mc

,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,фарпост работа владивосток,NaN,NaN,NaN,NaN,[общая фраза]
1,кофейни вакансии,NaN,NaN,NaN,NaN,[общая фраза]
2,работа разнорабочие часовой,NaN,[рабочий],NaN,NaN,NaN
3,личный водитель на день,[на неполный день],[водитель],NaN,NaN,NaN
4,работа от работодателя персональный водитель,NaN,[водитель],NaN,NaN,NaN
...,...,...,...,...,...,...
14240,япония вакансии,NaN,NaN,NaN,NaN,[общая фраза]
14241,япония работа,NaN,NaN,NaN,NaN,[общая фраза]
14242,японский язык работа,NaN,NaN,NaN,NaN,[общая фраза]
14243,яппи вакансии,NaN,NaN,NaN,NaN,[общая фраза]


In [86]:
# желательно использовать все же эмбеддинги
col = 'по должности-лемме'
df_new = df_mc[(~df_mc[col].isna())]
df_new[df_new[col].apply(lambda  x: len(x) > 1)][col].value_counts()

по должности-лемме
[уборщик,  горничная]                        148
[медсестра,  медбрат]                        127
[руководитель,  начальник]                    79
[домработница,  домработник]                  40
[гид,  экскурсовод]                            9
[декларант,  таможенный брокер]                7
[танцовщица,  танцовщик]                       6
[гувернантка,  гувернер]                       5
[сомелье,  кавист]                             3
[кастелянша,  заведующий бельевой]             3
[осветитель,  светотехник]                     2
[зооняня,  рабочий по уходу за животными]      1
[вальщик леса,  лесоруб]                       1
Name: count, dtype: int64

In [87]:
# желательно использовать все же эмбеддинги
col = 'по дополнительному признаку'
df_new = df_mc[(~df_mc[col].isna())]
df_new[df_new[col].apply(lambda  x: len(x) > 1)][col].value_counts()

по дополнительному признаку
[для пенсионеров, для женщин]       4
[для женщин, для пенсионеров]       3
[без опыта, для студентов]          3
[для женщин, без опыта]             3
[для пенсионеров, для мужчин]       2
[для мужчин, для женщин]            2
[без опыта, для женщин]             1
[для студентов, для школьников]     1
[для школьников, для студентов]     1
[для пенсионеров, для инвалидов]    1
[для студентов, без опыта]          1
Name: count, dtype: int64

In [88]:
# желательно использовать все же эмбеддинги
col = 'занятость'
df_new = df_mc[(~df_mc[col].isna())]
df_new[df_new[col].apply(lambda  x: len(x) > 1)][col].value_counts()

занятость
[на неполный день, на неполный день]                35
[по выходным, на неполный день]                     22
[подработка, по выходным]                           19
[удаленная, на дому]                                19
[вечерняя, подработка]                              13
[подработка, на неполный день]                      13
[подработка, на дому]                               12
[подработка, удаленная]                              7
[на дому, удаленная]                                 6
[подработка, по выходным, на неполный день]          5
[подработка, вечерняя]                               5
[ночная, подработка]                                 5
[удаленная, подработка]                              3
[удаленная, на неполный день]                        3
[на неполный день, удаленная]                        2
[, вахта]                                            2
[временная, подработка]                              2
[на дому, подработка]                                2


In [89]:
df[df.index.isin(df_new[df_new[col].apply(lambda x: x[0] == "")].index)]

,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
5197,охрана дома вахта,",Вахта",Вахтер,NaN,NaN,NaN
5198,охрана загородных домов вахта,",Вахта",Вахтер,NaN,NaN,NaN
6401,дом уборка подработка,",Подработка","Уборщик, горничная",NaN,NaN,NaN


In [90]:
df[df[col] == ",Вахта"]

,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
5197,охрана дома вахта,",Вахта",Вахтер,NaN,NaN,NaN
5198,охрана загородных домов вахта,",Вахта",Вахтер,NaN,NaN,NaN


План:
1. Разбить на несколько меток `занятость` и `по дополнительному признаку`
2. Избавиться от аномалий в виде запятых в начале текста...

## Составление списка меток для занятости и доп.признака

In [91]:
import re

In [92]:
df_mc = df.copy()
for col in ["занятость", "по дополнительному признаку"]:
    col_index = df_mc[~df_mc[col].isna()].index
    temp_col = f"temp_{col}"
    # df_col[temp_col] = df_col[col].str.strip().str.lower().str.split(',')
    for ind in col_index:
        descr = df_mc[col].iloc[ind]
        item_list = descr.strip(",. ").lower().split(',')
        clear_list = [item.strip(',. ') for item in item_list if item != ""]
        df_mc.loc[ind, col] = clear_list
display(
    df_mc["занятость"].explode().value_counts(), 
    df_mc["по дополнительному признаку"].explode().value_counts(), 
)

занятость
подработка          435
вахта               407
удаленная           302
на неполный день    212
на дому             181
по выходным          69
ночная               65
вечерняя             52
временная            14
посменная             5
дневная               4
посуточная            3
Name: count, dtype: int64

по дополнительному признаку
без опыта          469
для женщин         137
для студентов      104
для пенсионеров     63
для школьников      43
для мужчин          38
для инвалидов       17
Name: count, dtype: int64

### Приведение к норм виду других меток

In [93]:
one_class_cols = ["по должности-лемме", "по условиям"]
df_mc[one_class_cols] = df_mc[one_class_cols].apply(
    lambda x: x.str.lower()
)

In [94]:
df_mc[one_class_cols[0]].value_counts()

по должности-лемме
водитель         1010
курьер            434
помощник          244
вахтер            225
модератор         223
                 ... 
дилер               1
звукорежиссер       1
конюх               1
косильщик           1
чистильщик          1
Name: count, Length: 297, dtype: int64

In [96]:
df_mc[one_class_cols[1]].value_counts()

по условиям
с ежедневной оплатой    374
с проживанием           192
Name: count, dtype: int64

In [100]:
df_mc['общие фразы'] = df_mc['общие фразы'].fillna(0).map({"общая фраза": 1, 0: 0})

In [105]:
df_mc = df_mc.fillna("NaN")
df_mc

,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,фарпост работа владивосток,NaN,NaN,NaN,NaN,1
1,кофейни вакансии,NaN,NaN,NaN,NaN,1
2,работа разнорабочие часовой,NaN,рабочий,NaN,NaN,0
3,личный водитель на день,[на неполный день],водитель,NaN,NaN,0
4,работа от работодателя персональный водитель,NaN,водитель,NaN,NaN,0
...,...,...,...,...,...,...
14240,япония вакансии,NaN,NaN,NaN,NaN,1
14241,япония работа,NaN,NaN,NaN,NaN,1
14242,японский язык работа,NaN,NaN,NaN,NaN,1
14243,яппи вакансии,NaN,NaN,NaN,NaN,1


In [106]:
df_mc.to_csv("data/clear_dataset.csv", index=False)